In [ ]:
# %env CMAKE_ARGS="-DLLAMA_CUBLAS=on" 
# %env FORCE_CMAKE=1
# %pip install --upgrade --quiet  llama-cpp-python 

In [2]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.llms import LlamaCpp

llm = LlamaCpp(
    model_path="model\mistral-7b-instruct-v0.2.Q3_K_S.gguf",
    n_gpu_layers=1,
    n_batch=512,
    n_ctx=2048,
    f16_kv=True,
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
    verbose=True,
)

llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from model\mistral-7b-instruct-v0.2.Q3_K_S.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:            

In [3]:
from langchain.vectorstores import Chroma
from langchain.document_loaders import DirectoryLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceInstructEmbeddings

# loader

In [4]:
file_loader = DirectoryLoader("Document", glob="./*.pdf", loader_cls=PyPDFLoader)
documents = file_loader.load()
print(documents)

[Document(page_content='Bylaws\nArticle I. Gener al\n1. The Executive Committee shall be selec ted by the nomination \x00le by the members of the last\nyear’s Volunteer Team (as on Dec 31), pr eferably in Jan. The Ex ecutiv e Committee comprises of:\na. Chairperson\nb. Vice Chairperson\nc. Secr etary\nd. Treasur er\ne. W ebmaster\n2. \xa0A Volunteer Team of 10-12 members shall be elected for which election shall be conducted as\nfar as possible in Jan every year. This team will be subjected to expand twice in a year to include\nnewly joined activ e members in the team (selected b y curr ent Ex ecutiv e Committee members).\n3. Term of each Ex ecutiv e Committee and the V olunteer Team shall be at least one calendar y ear.\nArticle II. Quali\x00cations for being selec ted to an\nexecutiv e o\x00cer position\n1. All pr ospectiv e executiv e o\x00cers must be an IEEE Student member .\n2. All pr ospectiv e executiv e o\x00cers must be a member of the last V olunteer Team as on Dec 31.\n3. A

# split 

In [9]:
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=20,chunk_overlap=0)

In [10]:
# texts = text_splitter.split_documents(documents)

In [1]:
# instructor_embedding = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-large", model_kwargs={"device":"cuda"})

In [11]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings()

In [12]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(documents)
vector = FAISS.from_documents(documents, embeddings)

ValueError: Error raised by inference endpoint: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/embeddings (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001B731E784F0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [ ]:
persist_directory = "handbook_db"

# for loading the database

In [ ]:
# vector_db = Chroma(persist_directory=persist_directory, embedding_function=instructor_embedding)

In [ ]:
vector_db = Chroma.from_documents(documents=texts,                        
                                  embedding=instructor_embedding,                          
                                  persist_directory=persist_directory)
vector_db.persist()

# retriever

In [ ]:
retriever = vector_db.as_retriever(search_kwargs={"k":3})

# Context

In [ ]:
question='what are the rules for onboarding process for the new employee'
docs_result = retriever.get_relevant_documents(question)